In [ ]:
import pandas as pd
import numpy as np
import os
import string
import json
from typing import Optional

In [ ]:
def create_respondents_jsonl(df: pd.DataFrame, demographic_columns: list[str], demographic_labels: list[str]) -> None:
    """Create a JSONL file of respondents with their demographic data.

    Args:
        df (pd.DataFrame): DataFrame containing the respondents' data
        demographics (list[str]): List of demographic columns to include in the JSONL file
    """
    for c in demographic_columns:
        df[c] = df[c].astype(str).str.replace('_x000D_', '', regex=False).str.encode("ascii", "ignore").str.decode("ascii")
    for c in demographic_columns:
        df[c] = df[c].apply(lambda x: x.split(","))
    df.rename(columns=dict(zip(demographic_columns, demographic_labels)), inplace=True)
    df["demographic_data"] = df[demographic_labels].to_dict(orient="records")
    df[['themefinder_id',"demographic_data"]].to_json("inputs/respondents.jsonl", orient="records", lines=True)

def save_demographic_data(responses_df, question_understanding_path: str):
    demographic_info = pd.read_excel(question_understanding_path, sheet_name="Demographic", skiprows=3)
    
    demographic_questions = demographic_info[demographic_info.columns[0]].tolist()
    demographic_labels = demographic_info[demographic_info.columns[1]].tolist()
    demographic_labels = [label.replace('/', '-') for label in demographic_labels]

    for c in demographic_questions:
        responses_df[c] = responses_df[c].fillna("Not Provided")
        # AD HOC: check data before doing following
        responses_df[c] = responses_df[c].apply(lambda x: "Other" if isinstance(x, str) and "Other" in x else x)
    create_respondents_jsonl(responses_df, demographic_questions, demographic_labels)


def create_open_question_inputs(
    df: pd.DataFrame,
    open_questions: list[dict],
    characters_to_remove = ["/", "\\", '- Text', '_x000D_'],
    sample_size: Optional[int] = None
) -> None:
    for question in open_questions:
        q_num = question['question_number']
        question_col = question['column_name']
        q_dir = f"inputs/question_part_{q_num}"
        os.makedirs(q_dir, exist_ok=True)

        question_string = question['question_text']

        question_answers = df[['themefinder_id', question_col]].dropna()
        if sample_size is not None and sample_size < len(question_answers):
            question_answers = question_answers.sample(sample_size)

        for bad_string in characters_to_remove:
            question_answers[question_col] = question_answers[question_col].apply(lambda x: x.replace(bad_string, " "))
        
        question_answers[question_col] = question_answers[question_col].astype(str).str.encode("ascii", "ignore").str.decode("ascii")

        question_answers.columns = ['themefinder_id', 'text']

        question_answers[['themefinder_id', 'text']].to_json(os.path.join(q_dir, 'responses.jsonl'), orient='records', lines=True)

        question_data = {
            "question_number": q_num,
            "question_text": question_string,
            "has_free_text": True
        }

        with open(os.path.join(q_dir, 'question.json'), 'w') as f:
            json.dump(question_data, f, indent=4)

def save_open_questions(responses_df, question_understanding_path: str):
    question_info = pd.read_excel(question_understanding_path, sheet_name="Open questions", skiprows=3)
    
    question_info.columns = ["column_name", 'question_number', "question_text"]

    # remove questions with no answers
    only_nans = responses_df[question_info['column_name'].tolist()].isna().all()
    column_names_with_only_nans = only_nans[only_nans].index.tolist()
    question_info = question_info[~question_info['column_name'].isin(column_names_with_only_nans)]

    # Ensure question numbers are ints
    question_info['question_number'] = question_info['question_number'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
    if not question_info['question_number'].is_unique:
        raise AssertionError("Non-unique values found in 'question_number' column")

    create_open_question_inputs(responses_df, question_info.to_dict(orient="records"))


def create_hybrid_question_inputs(
    df: pd.DataFrame,
    hybrid_questions: list[dict],
    characters_to_remove = ["/", "\\", '- Text', '_x000D_'],
    sample_size: Optional[int] = None
) -> None:
    for question in hybrid_questions:
        q_num = question['question_number']
        q_dir = f"inputs/question_part_{q_num}"
        closed_col = question['closed_column']
        open_col = question['open_column']
        question_string = question['question_text']
        os.makedirs(q_dir, exist_ok=True)

        question_answers = df[['themefinder_id'] + [closed_col, open_col]].dropna(subset=[closed_col, open_col], how='all')

        if sample_size is not None and sample_size < len(question_answers):
            question_answers = question_answers.sample(sample_size)

        question_answers[closed_col] = question_answers[closed_col].fillna('Not Provided')
        question_answers[open_col] = question_answers[open_col].fillna('Not Provided')

        question_answers[closed_col] = question_answers[closed_col].astype(str).str.encode("ascii", "ignore").str.decode("ascii")
        question_answers[open_col] = question_answers[open_col].astype(str).str.encode("ascii", "ignore").str.decode("ascii")

        for bad_string in characters_to_remove:
            question_answers[closed_col] = question_answers[closed_col].apply(lambda x: x.replace(bad_string, " "))
            question_answers[open_col] = question_answers[open_col].apply(lambda x: x.replace(bad_string, " "))

        question_answers[closed_col] = question_answers[closed_col].apply(lambda x: x.split(","))

        question_answers.rename(columns={closed_col: 'options', open_col: 'text'}, inplace=True)

        question_answers[['themefinder_id','options']].to_json(os.path.join(q_dir, 'multi_choice.jsonl'), orient='records', lines=True)
        question_answers[['themefinder_id', 'text']].to_json(os.path.join(q_dir, 'responses.jsonl'), orient='records', lines=True)

        question_data = {
            "question_number": q_num,
            "question_text": question_string,
            "has_free_text": True,
            "multi_choice_options": list(set([item for sublist in question_answers['options'] for item in sublist])),
        }

        with open(os.path.join(q_dir, 'question.json'), 'w') as f:
            json.dump(question_data, f, indent=4)

def save_hybrid_questions(responses_df, question_understanding_path: str):
    question_info = pd.read_excel(question_understanding_path, sheet_name="Hybrid questions", skiprows=3)

    question_info.columns = ["closed_column",  "question_number", "question_text", "open_column"]

    # Ensure question numbers are ints
    question_info['question_number'] = question_info['question_number'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
    if not question_info['question_number'].is_unique:
        raise AssertionError("Non-unique values found in 'question_number' column")

    create_hybrid_question_inputs(responses_df, question_info.to_dict(orient="records"))


def create_closed_question_inputs(
    df: pd.DataFrame,
    closed_questions: list[dict],
    characters_to_remove = ["/", "\\", '- Text', '_x000D_'],
    sample_size: Optional[int] = None
) -> None:
    for question in closed_questions:
        q_num = question['question_number']
        question_col = question['column_name']
        q_dir = f"inputs/question_part_{q_num}"
        os.makedirs(q_dir, exist_ok=True)

        question_string = question['question_text']

        question_answers = df[['themefinder_id', question_col]].dropna()
        if sample_size is not None:
            question_answers = question_answers.sample(sample_size)

        question_answers[question_col] = question_answers[question_col].astype(str).str.encode("ascii", "ignore").str.decode("ascii")
        for bad_string in characters_to_remove:
            question_answers[question_col] = question_answers[question_col].apply(lambda x: x.replace(bad_string, " "))
        
        
        
        question_answers[question_col] = question_answers[question_col].apply(lambda x: x.split(","))

        question_answers.columns = ['themefinder_id', 'options']

        question_answers[['themefinder_id', 'options']].to_json(os.path.join(q_dir, 'multi_choice.jsonl'), orient='records', lines=True)

        question_data = {
            "question_number": q_num,
            "question_text": question_string,
            "has_free_text": False,
            "multi_choice_options": list(set([item for sublist in question_answers['options'] for item in sublist])),
        }

        with open(os.path.join(q_dir, 'question.json'), 'w') as f:
            json.dump(question_data, f, indent=4)

def save_closed_questions(responses_df, question_understanding_path: str):
    question_info = pd.read_excel(question_understanding_path, sheet_name="Multiple Choice", skiprows=3)

    question_info.columns = ["column_name", "question_number", "question_text"]

    # Ensure question numbers are ints
    question_info['question_number'] = question_info['question_number'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
    if not question_info['question_number'].is_unique:
        raise AssertionError("Non-unique values found in 'question_number' column")

    create_closed_question_inputs(responses_df, question_info.to_dict(orient="records"))

In [ ]:
# Utils

def get_excel_column_name(n):
    """Convert number to Excel column name (e.g., 0->A, 25->Z, 26->AA, 27->AB, ...)"""
    result = ""
    n += 1  # 1-based
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

# Load data

In [ ]:
responses_df = pd.read_excel("raw_data/responses_output_cleaned.xlsx", header=0)
responses_df.columns = [get_excel_column_name(i) for i in range(len(responses_df.columns))]
os.makedirs("inputs", exist_ok=True)
responses_df['themefinder_id'] = range(1, len(responses_df) + 1)

# Save Demographics

In [ ]:
save_demographic_data(responses_df, "question_understanding_path")

# Save Open Questions

In [ ]:
save_open_questions(responses_df, "question_understanding_path")

# Save Hybrid Questions

In [ ]:
save_hybrid_questions(responses_df, "question_understanding_path")

# Save Closed Questions

In [ ]:
save_closed_questions(responses_df, "question_understanding_path")